In [ ]:
import random
from functools import reduce
import operator
import qkit.gui.notebook.Progress_Bar as pb
import h5py

In [ ]:
def random_pick(some_list,probabilities):
    x = random.uniform(0,1)
    cumulative_probability=0.0
    for item,item_probability in zip(some_list,probabilities):
        cumulative_probability+=item_probability
        if x < cumulative_probability:
            break
    return item

def bred(pulse_generation, test_results):
    p_c = 0.7
    p_m = 0.005
    
    # unify test_results
    test_results = test_results - min(test_results)
    total = sum(test_results)
    probability = test_results/total
    
    #probability = [np.where(test_results[i] > 60, 1, 0) for i in range(10)]
    #probability = probability/sum(probability)
    
    item = [n for n in range(len(pulse_generation))]
    next_generation = pulse_generation
    for i in range(int(len(pulse_generation)/2)):
        # choose parents
        p_1 = random_pick(item, probability)
        p_2 = random_pick(item, probability)
        if random.uniform(0,1) < p_c:
            cross_point = random.randint(0,len(pulse_generation[i])-1)
            next_generation[2*i] = pulse_generation[p_1][0:cross_point+1]+pulse_generation[p_2][cross_point+1:]
            next_generation[2*i+1] = pulse_generation[p_2][0:cross_point+1]+pulse_generation[p_1][cross_point+1:]
        else:
            next_generation[2*i] = pulse_generation[p_1]
            next_generation[2*i+1] = pulse_generation[p_2]
    next_generation = mutation(next_generation, p_m)
    return next_generation

def mutation(next_generation, p_m):
    for j in range(len(next_generation)):
        for k in range(len(next_generation[j])):
            if random.uniform(0,1) < p_m:
                next_generation[j][k] = 1 - next_generation[j][k]
    return next_generation

def test_pulse(pulse_sequence):
    points = 100
    func = binary_digits(np.linspace(0,1,points), pulse_sequence) # generate function handle
    shape = qi_pulse.Shape('GA_shape', func) # * qi_pulse.Shape('rect', lambda x: np.where(x>=0 and x<1, 1, 0)) # define shape
    
    with QiJob() as readout_clouds: # define jobs using shape above
        q = QiCells(1)
        Pulse = QiPulse(
            length = q[0]['rec_pulse'],
            shape = shape,
            frequency = 61472222.222222224
        )
        ql.jobs.PlayReadout(q[0], Pulse)
        ql.jobs.Recording(q[0],q[0]['rec_length'],q[0]['rec_offset'],save_to="result")
        ql.jobs.Thermalize(q[0])
        
        ql.jobs.PiPulse(q[0])
        ql.jobs.PlayReadout(q[0], Pulse)
        ql.jobs.Recording(q[0],q[0]['rec_length'],q[0]['rec_offset'],save_to="result")
        ql.jobs.Thermalize(q[0])
        clarity = measure_clarity(readout_clouds) # change the goodness to make better pulses stand out
    return clarity

def binary_digits(x_val, y_val):
    resolution = len(x_val)
    unit = 1/resolution
    bits = len(y_val)/resolution
    pulse_value = [0 for j in range(resolution)]
    for i in range(resolution):
        binary = ''
        for j in range(int(bits)):
            binary = binary+str(y_val[int(i*bits+j)])
        pulse_value[i] = int(binary, 2)/(pow(2,bits)-0.99)
    def return_fun(x):
        if x< 0 or x >= 1:
            return 0.0
        else:
            return pulse_value[int(x/unit)]
    return return_fun

def digitize_function(func, points, digits):
    unit = 1/pow(2, digits)
    individual = []
    x = np.linspace(0,1,points)
    for i in range(points):
        individual = individual+[0 for k in range(digits)]
        bin_str = bin(int(func(x[i])/unit))
        j = -1
        while bin_str[j] != 'b':
            # print(j)
            individual[j] = int(bin_str[j])
            j = j-1
    return individual

In [ ]:
# fancy shape initialization
pulse_generation = [[0 for i in range(600)] for j in range(20)]

# rectangular
rectangular_individual = [1,1,1,1,1,1]*100
num_rect = 5
pulse_generation[0:num_rect] = [rectangular_individual for i in range(num_rect)]

# gaussian
gauss_individual = digitize_function(lambda x: np.exp(-0.5 * np.power((x - 0.5) / 0.166, 2.0)), 100, 6)
num_gauss = 5
pulse_generation[num_rect:num_rect+num_gauss] = [gauss_individual for i in range(num_gauss)]

# exponential
r_individual = digitize_function(lambda x: np.exp(-3*x)-0.01, 100, 6)
num_r = 5
pulse_generation[num_rect+num_gauss:num_rect+num_gauss+num_r] = [r_individual for i in range(num_r)]

# step rectangular
h_individual = [1,1,1,1,1,1]*30+[0,0,0,0,0,0]*70
num_h = 5
pulse_generation[num_rect+num_gauss+num_r:num_rect+num_gauss+num_r+num_h] = [h_individual for i in range(num_h)]

# plot to see
for i in [2, 7, 12, 17]:
    func_se = binary_digits(np.linspace(0,1,100), pulse_generation[i])
    x = np.linspace(0,1,1000)
    y = [func_se(x[i]) for i in range(len(x))]
    plt.plot(x, y)

plt.savefig(folder_name+'First_Generation')
plt.show()
print('First generation ready')

In [ ]:
# Initialize !

att_old = frontend_box.get_attenuation_rf_readout()
print('attenuation before experiment= ', att_old)
frontend_box.set_attenuation_rf_readout(4.0)
print('attenuation during experiment= ', frontend_box.get_attenuation_rf_readout())


num_generation = 20
points = 600


next_generation = list(np.zeros((num_generation, points)))
test_results = list(np.zeros(num_generation))
# test_results_new = test_results

# test random
# plt.plot(pulse_generation[4], '+')
# plt.show()

for i in range(len(test_results)):
    test_results[i] = test_pulse(pulse_generation[i])
plt.plot(test_results)
plt.show()

# Record Data During GA
iteration = 0
Goodness = []
Best = []
Best_individual_index = []
T = np.zeros([51, num_generation, points])

p = pb.Progress_Bar(51)
while iteration <= 50:
    if iteration%1 == 0:
        Goodness.append(sum(test_results)/num_generation)
        Best.append(max(test_results))
        Best_individual_index.append(np.argmax(test_results))
        T[iteration] = pulse_generation
    
    #print(len(pulse_generation))
    next_generation = bred(pulse_generation, test_results)
    #print(len(pulse_generation))
    #print(next_generation)
    for i in range(len(test_results)):
        #print(len(next_generation[3]))
        test_results[i] = test_pulse(next_generation[i])
    pulse_generation = next_generation
    iteration = iteration + 1
    p.iterate()

K = list(range(0,51,1))
plt.plot(K, Goodness, 'b')
plt.plot(K, Best, 'r')
plt.xlabel('Generation')
plt.ylabel('Goodness')
plt.show()

frontend_box.set_attenuation_rf_readout(att_old)
print(frontend_box.get_attenuation_rf_readout())